## PoH + TRM Latent Recursion vs Official TRM on Maze 30×30 (Colab)

This notebook prepares the Maze-Hard dataset, and runs either:
- Official TRM recipe from TinyRecursiveModels (single-GPU A100)
- Your PoH + TRM-style latent recursion runner in this repo

Reference: TinyRecursiveModels [repo](https://github.com/SamsungSAILMontreal/TinyRecursiveModels)



In [ ]:
#@title Install base dependencies
%pip -q install --upgrade pip wheel setuptools
%pip -q install tqdm huggingface_hub



In [ ]:
#@title Option A: Official TRM (TinyRecursiveModels)
%%bash
set -e
# Clone and install
rm -rf /content/TinyRecursiveModels
git clone https://github.com/SamsungSAILMontreal/TinyRecursiveModels.git /content/TinyRecursiveModels
cd /content/TinyRecursiveModels
pip -q install --pre --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126
pip -q install -r requirements.txt
pip -q install --no-cache-dir --no-build-isolation adam-atan2

# Build Maze-Hard dataset (writes to data/maze-30x30-hard-1k)
python dataset/build_maze_dataset.py



In [ ]:
#@title Run TRM Maze-Hard (single A100)
%%bash
set -e
cd /content/TinyRecursiveModels
run_name="pretrain_att_maze30x30_a100_1gpu"
python pretrain.py \
  arch=trm \
  data_paths="[data/maze-30x30-hard-1k]" \
  evaluators="[]" \
  epochs=50000 eval_interval=5000 \
  lr=1e-4 puzzle_emb_lr=1e-4 weight_decay=1.0 puzzle_emb_weight_decay=1.0 \
  arch.L_layers=2 \
  arch.H_cycles=3 arch.L_cycles=4 \
  +run_name=${run_name} ema=True



In [ ]:
#@title Option B: PoH + TRM-latent (this repo)
%%bash
set -e
rm -rf /content/PoT
git clone https://github.com/Eran-BA/PoT.git /content/PoT
cd /content/PoT
pip -q install -r requirements.txt
pip -q install tqdm huggingface_hub

python - << 'PY'
import os, csv, json, numpy as np
from huggingface_hub import hf_hub_download
from tqdm import tqdm
output_dir = '/content/PoT/vendor/hrm/data/maze-30x30-hard-1k'
os.makedirs(output_dir, exist_ok=True)
CHARSET = "# SGo"
repo = "sapientinc/maze-30x30-hard-1k"
for split in ['train','test']:
    split_dir = os.path.join(output_dir, split)
    os.makedirs(split_dir, exist_ok=True)
    csv_file = hf_hub_download(repo, f"{split}.csv", repo_type="dataset")
    inputs, labels = [], []
    with open(csv_file, 'r') as f:
        reader = csv.reader(f); next(reader)
        for source, q, a, rating in tqdm(reader):
            n = int(len(q)**0.5)
            inputs.append(np.frombuffer(q.encode(), dtype=np.uint8).reshape(n,n))
            labels.append(np.frombuffer(a.encode(), dtype=np.uint8).reshape(n,n))
    char2id = np.zeros(256, np.uint8)
    for i,c in enumerate("\0" + CHARSET):
        char2id[ord(c)] = i
    X = np.vstack([char2id[x.reshape(-1)] for x in inputs])
    Y = np.vstack([char2id[y.reshape(-1)] for y in labels])
    np.save(os.path.join(split_dir,'all__inputs.npy'), X)
    np.save(os.path.join(split_dir,'all__labels.npy'), Y)
    with open(os.path.join(split_dir,'dataset.json'),'w') as f:
        json.dump({'seq_len':900,'vocab_size':6}, f)
print('✓ Dataset ready at', output_dir)
PY



In [ ]:
#@title Run PoH + TRM-latent (~35M params)
%%bash
set -e
cd /content/PoT
python -u experiments/maze_grid2grid_hrm.py \
  --data-dir vendor/hrm/data/maze-30x30-hard-1k \
  --model poh \
  --d-model 512 --n-heads 8 --n-layers 6 --d-ff 2048 \
  --T 4 --batch-size 12 \
  --max-epochs 5000 --patience 50 \
  --lr 1e-4 --puzzle-emb-lr 5e-4 \
  --weight-decay 1.0 \
  --num-puzzles 1000 --puzzle-emb-dim 256 \
  --max-halting-steps 16 \
  --latent-len 16 --latent-k 3 \
  --output experiments/results/poh_trm_latent_35m --seed 123



### Notes
- TRM reference run mirrors the official recipe but on a single GPU (A100). For multi-GPU, replace the python call with torchrun (nproc-per-node=4).
- Our PoH runner uses adaptive halting and latent recursion. Start with puzzle-emb weight_decay=0.0; ablate 0.1 and 1.0 if you want strict TRM regularization.
- Source: TinyRecursiveModels README (Maze-Hard): https://github.com/SamsungSAILMontreal/TinyRecursiveModels

